![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 

## Company's description 📇

<a href="https://www.kayak.com" target="_blank">Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href="https://www.bookingholdings.com/" target="_blank">Booking Holdings</a> which now holds: 

* <a href="https://booking.com/" target="_blank">Booking.com</a>
* <a href="https://kayak.com/" target="_blank">Kayak</a>
* <a href="https://www.priceline.com/" target="_blank">Priceline</a>
* <a href="https://www.agoda.com/" target="_blank">Agoda</a>
* <a href="https://Rentalcars.com/" target="_blank">RentalCars</a>
* <a href="https://www.opentable.com/" target="_blank">OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">One Week In.com</a> here are the top-35 cities to visit in France: 

```python 
["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]
```

Your team should focus **only on the above cities for your project**. 


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)

Step 1 - API

In [6]:
import pandas as pd 
import requests
import json
import html_to_json

from keys import open_weather_api_key

In [7]:
top_cities_fr=["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]


In [8]:
#API for gps coordinates / Test API
location_gps="https://nominatim.openstreetmap.org/search"
headers = {"User-Agent": 'Jedha FMS' }
payload={'q':'Paris','format':'json'}
r_gps=requests.get(url=location_gps,params=payload,headers=headers).json()

In [9]:
#API for gps coordinates / Test API
print(type(r_gps))

ville=r_gps[0]['name']
lat1=r_gps[0]['boundingbox'][0]
lat2=r_gps[0]['boundingbox'][1]
lon1=r_gps[0]['boundingbox'][2]
lon2=r_gps[0]['boundingbox'][3]

df_col=pd.DataFrame(columns=['Ville','Latitude 1','Latitude 2','Longitude 1', 'Longitude 2'])
list_gps = [ville, lat1, lat2, lon1, lon2]

df_col.loc[0] = list_gps
df_col




<class 'list'>


,Ville,Latitude 1,Latitude 2,Longitude 1,Longitude 2
0,Paris,48.8155755,48.9021560,2.2241220,2.4697602


In [10]:
#API for gps coordinates / top cities fr

df_gps=pd.DataFrame(columns=['Ville','Latitude 1','Latitude 2','Longitude 1', 'Longitude 2'])

for i in top_cities_fr:
    location_gps="https://nominatim.openstreetmap.org/search"
    headers = {"User-Agent": 'Jedha FMS' }
    payload={'q':i,'format':'json'}
    r_gps=requests.get(url=location_gps,params=payload,headers=headers).json()
    
    ville=r_gps[0]['name']
    lat1=r_gps[0]['boundingbox'][0]
    lat2=r_gps[0]['boundingbox'][1]
    lon1=r_gps[0]['boundingbox'][2]
    lon2=r_gps[0]['boundingbox'][3]
    
    list_gps = [ville, lat1, lat2, lon1, lon2]
    
    index_i=top_cities_fr.index(i)
    df_gps.loc[index_i] = list_gps

df_gps


,Ville,Latitude 1,Latitude 2,Longitude 1,Longitude 2
0,Mont Saint-Michel,48.6349172,48.6370310,-1.5133292,-1.5094796
1,Saint-Malo,48.5979853,48.6949736,-2.0765246,-1.9367259
2,Bayeux,49.2608124,49.2934736,-0.7275523,-0.6757378
3,Le Havre,49.4516697,49.5401463,0.0667992,0.1955556
4,Rouen,49.4172001,49.4652601,1.0300648,1.1521157
5,Paris,48.8155755,48.9021560,2.2241220,2.4697602
6,Amiens,49.8468370,49.9505487,2.2235574,2.3457767
7,Lille,50.6008264,50.6612596,2.9679677,3.1257250
8,Strasbourg,48.4918610,48.6461896,7.6881371,7.8360646
9,Château du Haut-Kœnigsbourg,48.2489931,48.2498214,7.3425172,7.3462060


In [11]:
#Transfer results into a CSV file
df_gps.to_csv('DF_GPS')


In [15]:
lat=df_gps['Latitude 1']
lon=df_gps['Longitude 1']

coordinates=lat+','+lon

In [46]:
#API for weather infos / Test API - Paris

location_weather='http://api.openweathermap.org/data/2.5/forecast?'
payload={'lat':'48.8155755','lon':'2.2241220','APPID':open_weather_api_key,'units':'metric','lang':'fr','cnt':40}
r_weather=requests.get(location_weather,params=payload).json()

r_weather['list'][]

{'dt': 1731466800,
 'main': {'temp': 8.06,
  'feels_like': 5.7,
  'temp_min': 7.52,
  'temp_max': 8.06,
  'pressure': 1030,
  'sea_level': 1030,
  'grnd_level': 1018,
  'humidity': 80,
  'temp_kf': 0.54},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'couvert',
   'icon': '04n'}],
 'clouds': {'all': 88},
 'wind': {'speed': 3.84, 'deg': 43, 'gust': 8.78},
 'visibility': 10000,
 'pop': 0,
 'sys': {'pod': 'n'},
 'dt_txt': '2024-11-13 03:00:00'}

In [35]:
lat=df_gps['Latitude 1']
lon=df_gps['Longitude 1']
lon

0     -1.5133292
1     -2.0765246
2     -0.7275523
3      0.0667992
4      1.0300648
5      2.2241220
6      2.2235574
7      2.9679677
8      7.6881371
9      7.3425172
10     7.3154803
11     7.2412792
12     5.9409668
13     4.9624434
14     6.0484121
15     5.6776059
16     4.7718134
17     6.3285116
18     6.2716142
19     5.5063193
20     5.2286312
21     5.2694745
22     4.7396309
23     4.3852299
24     4.2356708
25     4.0760703
26     4.2301364
27     3.0495074
28     2.2615684
29     0.8267506
30     1.3503956
31     1.2865419
32    -1.5773906
33    -1.5048438
34    -1.2419231
Name: Longitude 1, dtype: object

In [59]:
lat=df_gps['Latitude 1']
lon=df_gps['Longitude 1']
ville=df_gps['Ville']

weather_dict={}
keys=['Ville','Date', 'Feels like','Temp min','Temp max','Humidity','Description']

ville_list=[]
date_list=[]
feels_like_list=[]
temp_min_list=[]
temp_max_list=[]
humidity_list=[]
description_list=[]

for i in range(len(lat)):

    location_weather='http://api.openweathermap.org/data/2.5/forecast?'
    payload={'lat': lat[i],'lon':lon[i],'APPID':open_weather_api_key,'units':'metric','lang':'fr','cnt':40}
    r_weather=requests.get(location_weather,params=payload).json()
    
    for e in range (0,40):
        date=r_weather['list'][e]['dt_txt']
        feels_like=r_weather['list'][e]['main']['feels_like']
        temp_min=r_weather['list'][e]['main']['temp_min']
        temp_max=r_weather['list'][e]['main']['temp_max']
        humidity=r_weather['list'][e]['main']['humidity']
        description=r_weather['list'][e]['weather'][0]['description']
    
        ville_list.append(ville[i])
        date_list.append(date)
        feels_like_list.append(feels_like)
        temp_min_list.append(temp_min)
        temp_max_list.append(temp_max)
        humidity_list.append(humidity)
        description_list.append(description)

weather_dict['Ville']=ville_list
weather_dict['Date']=date_list
weather_dict['Feels like']=feels_like_list
weather_dict['Temp min']=temp_min_list
weather_dict['Temp max']=temp_max_list
weather_dict['Humidity']=humidity_list
weather_dict['Description']=description_list

df_weather=pd.DataFrame.from_dict(weather_dict)
df_weather

,Ville,Date,Feels like,Temp min,Temp max,Humidity,Description
0,Mont Saint-Michel,2024-11-13 00:00:00,4.60,6.54,7.49,88,ciel dégagé
1,Mont Saint-Michel,2024-11-13 03:00:00,3.62,5.99,6.65,90,peu nuageux
2,Mont Saint-Michel,2024-11-13 06:00:00,5.38,7.62,7.62,82,partiellement nuageux
3,Mont Saint-Michel,2024-11-13 09:00:00,4.56,7.60,7.60,85,couvert
4,Mont Saint-Michel,2024-11-13 12:00:00,9.23,10.36,10.36,68,couvert
...,...,...,...,...,...,...,...
1395,La Rochelle,2024-11-17 09:00:00,13.17,13.61,13.61,82,légère pluie
1396,La Rochelle,2024-11-17 12:00:00,13.55,13.89,13.89,85,légère pluie
1397,La Rochelle,2024-11-17 15:00:00,11.72,12.15,12.15,88,pluie modérée
1398,La Rochelle,2024-11-17 18:00:00,11.60,12.45,12.45,71,pluie modérée


In [60]:
df_weather.to_csv('DF_WEATHER')